# Feature Engineering for CNC ML Project

This notebook focuses on creating engineered features from the raw CNC data to improve machine learning model performance.

## Objectives
1. Apply data preprocessing pipeline
2. Create time-based features
3. Engineer performance metrics and ratios
4. Build operator and machine interaction features
5. Create rolling averages and trend features
6. Prepare features for modeling

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import mutual_info_regression
import warnings

# Import project modules
import sys
sys.path.append('../src')

from data.database import DatabaseManager
from data.preprocessing import DataPreprocessor
from utils.helpers import setup_logging, create_time_features

# Configuration
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

logger = setup_logging()

print("Feature Engineering Notebook - CNC ML Project")
print("=" * 50)

## 1. Data Loading and Initial Preprocessing

In [ ]:
# Load data
print("Loading data...")
db = DatabaseManager()
df_raw = db.get_all_data(limit=10000)  # Load 10k records for feature engineering

print(f"Loaded {len(df_raw)} records")
print(f"Columns: {list(df_raw.columns)}")
print(f"Date range: {df_raw['StartTime'].min()} to {df_raw['StartTime'].max()}")

In [ ]:
# Initialize and apply data preprocessor
print("Applying data preprocessing...")
preprocessor = DataPreprocessor()

# Validate and clean raw data
df_clean = preprocessor.validate_raw_data(df_raw)
print(f"After cleaning: {len(df_clean)} records ({len(df_raw) - len(df_clean)} removed)")

# Create derived features
df_features = preprocessor.create_derived_features(df_clean)
print(f"Features created. Dataset shape: {df_features.shape}")

# Display new features
new_features = set(df_features.columns) - set(df_raw.columns)
print(f"\nNew features created ({len(new_features)}):")
for feature in sorted(new_features):
    print(f"  - {feature}")

## 2. Advanced Time-Based Features

In [ ]:
# Create advanced time-based features
print("Creating advanced time-based features...")

# Sort data by machine and time for sequential features
df_features = df_features.sort_values(['machine', 'StartTime'])

# Time since last job on same machine
df_features['time_since_last_job'] = df_features.groupby('machine')['StartTime'].diff().dt.total_seconds()
df_features['time_since_last_job'] = df_features['time_since_last_job'].fillna(0)

# Job sequence number on machine
df_features['machine_job_sequence'] = df_features.groupby('machine').cumcount() + 1

# Time-based efficiency patterns
if 'hour' in df_features.columns:
    # Rush hour indicator (typically high production hours)
    rush_hours = [8, 9, 10, 14, 15, 16]  # Common production peak hours
    df_features['is_rush_hour'] = df_features['hour'].isin(rush_hours).astype(int)
    
    # Shift start/end indicators
    df_features['shift_start'] = df_features['hour'].isin([6, 14, 22]).astype(int)  # Common shift start times
    df_features['shift_end'] = df_features['hour'].isin([13, 21, 5]).astype(int)   # Common shift end times

# Weekly patterns
if 'day_of_week' in df_features.columns:
    # Week position (beginning, middle, end)
    df_features['week_position'] = df_features['day_of_week'].apply(
        lambda x: 'start' if x <= 1 else ('middle' if x <= 3 else 'end')
    )
    
    # Production day vs non-production day
    df_features['production_day'] = (df_features['day_of_week'] < 5).astype(int)  # Monday-Friday

print(f"Time-based features created. New shape: {df_features.shape}")

## 3. Performance Ratio Features

In [ ]:
# Create comprehensive performance ratio features
print("Creating performance ratio features...")

# Basic ratios (already created in preprocessing)
print(f"Efficiency range: {df_features['efficiency'].min():.3f} - {df_features['efficiency'].max():.3f}")

# Advanced performance ratios
# Productive time ratio (RunningTime / (RunningTime + SetupTime))
if 'SetupTime' in df_features.columns:
    df_features['productive_time_ratio'] = (
        df_features['RunningTime'] / 
        (df_features['RunningTime'] + df_features['SetupTime'] + 1)  # +1 to avoid division by zero
    )

# Downtime intensity (total_downtime / JobDuration)
df_features['downtime_intensity'] = df_features['total_downtime'] / (df_features['JobDuration'] + 1)

# Setup efficiency (1 / setup_time, normalized)
if 'SetupTime' in df_features.columns:
    df_features['setup_efficiency'] = 1 / (df_features['SetupTime'] / 1000 + 1)  # Normalized

# Parts production rate relative to time
if 'PartsProduced' in df_features.columns:
    df_features['parts_per_minute'] = df_features['PartsProduced'] * 60 / (df_features['RunningTime'] + 1)
    df_features['parts_per_minute'] = df_features['parts_per_minute'].replace([np.inf, -np.inf], 0)

print(f"Performance ratios created. Shape: {df_features.shape}")

# Display ratio statistics
ratio_features = ['efficiency', 'productive_time_ratio', 'downtime_intensity', 'setup_efficiency']
ratio_features = [f for f in ratio_features if f in df_features.columns]

print("\nPerformance Ratio Statistics:")
for feature in ratio_features:
    mean_val = df_features[feature].mean()
    std_val = df_features[feature].std()
    print(f"  {feature}: {mean_val:.3f} ± {std_val:.3f}")

## 4. Machine and Operator Interaction Features

In [ ]:
# Create machine-operator interaction features
print("Creating machine-operator interaction features...")

# Historical performance by machine
machine_historical = df_features.groupby('machine').agg({
    'efficiency': ['mean', 'std'],
    'JobDuration': 'mean',
    'SetupTime': 'mean',
    'total_downtime': 'mean'
})

# Flatten column names
machine_historical.columns = ['_'.join(col).strip() for col in machine_historical.columns]

# Map machine historical features to dataset
for col in machine_historical.columns:
    feature_name = f'machine_hist_{col}'
    df_features[feature_name] = df_features['machine'].map(machine_historical[col])

# Historical performance by operator (excluding 'Unknown')
if 'OperatorName' in df_features.columns:
    operator_data = df_features[df_features['OperatorName'] != 'Unknown']
    
    if len(operator_data) > 0:
        operator_historical = operator_data.groupby('OperatorName').agg({
            'efficiency': ['mean', 'std'],
            'SetupTime': 'mean',
            'machine': 'nunique',
            'PartNumber': 'nunique'
        })
        
        operator_historical.columns = ['_'.join(col).strip() for col in operator_historical.columns]
        
        # Map operator historical features
        for col in operator_historical.columns:
            feature_name = f'operator_hist_{col}'
            df_features[feature_name] = df_features['OperatorName'].map(operator_historical[col])
            df_features[feature_name] = df_features[feature_name].fillna(df_features[feature_name].median())

# Machine-operator combination experience
if 'OperatorName' in df_features.columns:
    df_features['machine_operator_jobs'] = df_features.groupby(['machine', 'OperatorName']).cumcount() + 1
    
    # Experience category
    df_features['experience_category'] = pd.cut(
        df_features['machine_operator_jobs'],
        bins=[0, 1, 5, 15, float('inf')],
        labels=['novice', 'beginner', 'experienced', 'expert']
    )

print(f"Interaction features created. Shape: {df_features.shape}")

## 5. Rolling Window Features

In [ ]:
# Create rolling window features for trend analysis
print("Creating rolling window features...")

# Sort data for proper rolling calculations
df_features = df_features.sort_values(['machine', 'OperatorName', 'StartTime'])

# Rolling features for machine performance
for window in [3, 5, 10]:
    # Machine efficiency trends
    df_features[f'machine_efficiency_roll_{window}'] = (
        df_features.groupby('machine')['efficiency']
        .rolling(window=window, min_periods=1)
        .mean()
        .reset_index(0, drop=True)
    )
    
    # Machine downtime trends
    df_features[f'machine_downtime_roll_{window}'] = (
        df_features.groupby('machine')['total_downtime']
        .rolling(window=window, min_periods=1)
        .mean()
        .reset_index(0, drop=True)
    )

# Rolling features for operator performance (if applicable)
if 'OperatorName' in df_features.columns:
    for window in [3, 5]:
        # Operator efficiency trends
        df_features[f'operator_efficiency_roll_{window}'] = (
            df_features.groupby(['OperatorName'])['efficiency']
            .rolling(window=window, min_periods=1)
            .mean()
            .reset_index([0], drop=True)
        )
        
        # Operator setup time trends
        if 'SetupTime' in df_features.columns:
            df_features[f'operator_setup_roll_{window}'] = (
                df_features.groupby(['OperatorName'])['SetupTime']
                .rolling(window=window, min_periods=1)
                .mean()
                .reset_index([0], drop=True)
            )

# Trend features (difference from rolling mean)
df_features['efficiency_vs_machine_trend'] = (
    df_features['efficiency'] - df_features['machine_efficiency_roll_5']
)

if 'operator_efficiency_roll_3' in df_features.columns:
    df_features['efficiency_vs_operator_trend'] = (
        df_features['efficiency'] - df_features['operator_efficiency_roll_3']
    )

print(f"Rolling features created. Shape: {df_features.shape}")

# Show trend feature statistics
trend_features = [col for col in df_features.columns if 'trend' in col or 'roll' in col]
print(f"\nCreated {len(trend_features)} trend/rolling features")

## 6. Part Complexity Features

In [ ]:
# Create part complexity and specialization features
print("Creating part complexity features...")

if 'PartNumber' in df_features.columns:
    # Filter out unknown parts
    part_data = df_features[df_features['PartNumber'] != 'Unknown']
    
    if len(part_data) > 0:
        # Part complexity based on average setup time
        part_complexity = part_data.groupby('PartNumber').agg({
            'SetupTime': 'mean',
            'JobDuration': 'mean',
            'efficiency': 'mean',
            'machine': 'nunique',
            'OperatorName': 'nunique'
        })
        
        part_complexity.columns = ['avg_setup_time', 'avg_job_duration', 'avg_efficiency', 'machines_used', 'operators_used']
        
        # Calculate complexity scores
        part_complexity['complexity_score'] = (
            part_complexity['avg_setup_time'] / part_complexity['avg_setup_time'].max()
        )
        
        part_complexity['versatility_score'] = (
            part_complexity['machines_used'] * part_complexity['operators_used']
        )
        
        # Map part features to main dataset
        for col in part_complexity.columns:
            feature_name = f'part_{col}'
            df_features[feature_name] = df_features['PartNumber'].map(part_complexity[col])
            # Fill missing values for 'Unknown' parts with median
            df_features[feature_name] = df_features[feature_name].fillna(df_features[feature_name].median())
        
        # Part frequency (how often it's produced)
        part_frequency = df_features['PartNumber'].value_counts()
        df_features['part_frequency'] = df_features['PartNumber'].map(part_frequency)
        df_features['part_frequency'] = df_features['part_frequency'].fillna(1)
        
        # Part popularity category
        df_features['part_popularity'] = pd.cut(
            df_features['part_frequency'],
            bins=[0, 1, 5, 20, float('inf')],
            labels=['rare', 'uncommon', 'common', 'frequent']
        )
        
        print(f"Part complexity features created for {len(part_complexity)} unique parts")
    else:
        print("No valid part data found for complexity analysis")
else:
    print("No PartNumber column found")

print(f"Current dataset shape: {df_features.shape}")

## 7. Anomaly and Quality Features

In [ ]:
# Create anomaly and quality indicator features
print("Creating anomaly and quality features...")

# Extreme value indicators
# Efficiency anomalies
efficiency_q95 = df_features['efficiency'].quantile(0.95)
efficiency_q05 = df_features['efficiency'].quantile(0.05)
df_features['efficiency_extreme_high'] = (df_features['efficiency'] > efficiency_q95).astype(int)
df_features['efficiency_extreme_low'] = (df_features['efficiency'] < efficiency_q05).astype(int)

# Duration anomalies
duration_q95 = df_features['JobDuration'].quantile(0.95)
duration_q05 = df_features['JobDuration'].quantile(0.05)
df_features['duration_extreme_long'] = (df_features['JobDuration'] > duration_q95 * 2).astype(int)
df_features['duration_extreme_short'] = (df_features['JobDuration'] < duration_q05 / 2).astype(int)

# Downtime anomalies
downtime_q95 = df_features['total_downtime'].quantile(0.95)
df_features['downtime_extreme_high'] = (df_features['total_downtime'] > downtime_q95 * 2).astype(int)

# Quality indicators
# Consistency indicators (based on rolling standard deviation)
df_features['efficiency_consistency'] = (
    df_features.groupby('machine')['efficiency']
    .rolling(window=5, min_periods=2)
    .std()
    .reset_index(0, drop=True)
    .fillna(0)
)

# Stability score (inverse of consistency for better interpretation)
df_features['stability_score'] = 1 / (1 + df_features['efficiency_consistency'])

# Performance deviation from expected
if 'machine_hist_efficiency_mean' in df_features.columns:
    df_features['performance_deviation'] = (
        abs(df_features['efficiency'] - df_features['machine_hist_efficiency_mean']) /
        (df_features['machine_hist_efficiency_std'] + 0.01)  # Normalized by std dev
    )

# Rush job indicator (jobs with very short time since last job)
df_features['rush_job'] = (df_features['time_since_last_job'] < 300).astype(int)  # Less than 5 minutes

print(f"Anomaly and quality features created. Shape: {df_features.shape}")

# Show anomaly statistics
anomaly_features = [col for col in df_features.columns if 'extreme' in col or 'rush' in col]
print("\nAnomaly Feature Statistics:")
for feature in anomaly_features:
    count = df_features[feature].sum()
    pct = (count / len(df_features)) * 100
    print(f"  {feature}: {count} ({pct:.1f}%)")

## 8. Feature Selection and Importance Analysis

In [ ]:
# Analyze feature importance and correlations
print("Analyzing feature importance...")

# Select numerical features for analysis
numerical_features = df_features.select_dtypes(include=[np.number]).columns.tolist()

# Remove target-like features and identifiers for feature selection
exclude_features = [
    'efficiency', 'RunningTime', 'JobDuration', 'PartsProduced',
    'StartTime', 'EndTime'  # These might be datetime but could appear as numbers
]

feature_candidates = [f for f in numerical_features if f not in exclude_features]

print(f"Analyzing {len(feature_candidates)} feature candidates for efficiency prediction")

# Calculate mutual information with efficiency (target)
if len(feature_candidates) > 0:
    # Prepare feature matrix (handle missing values)
    X = df_features[feature_candidates].fillna(df_features[feature_candidates].median())
    y = df_features['efficiency']
    
    # Calculate mutual information
    try:
        mi_scores = mutual_info_regression(X, y, random_state=42)
        
        # Create feature importance dataframe
        feature_importance = pd.DataFrame({
            'feature': feature_candidates,
            'mutual_info_score': mi_scores
        }).sort_values('mutual_info_score', ascending=False)
        
        print("\nTop 15 Most Important Features (by Mutual Information):")
        print(feature_importance.head(15))
        
        # Visualize top features
        plt.figure(figsize=(12, 8))
        top_features = feature_importance.head(20)
        plt.barh(range(len(top_features)), top_features['mutual_info_score'])
        plt.yticks(range(len(top_features)), top_features['feature'])
        plt.xlabel('Mutual Information Score')
        plt.title('Top 20 Feature Importance (Mutual Information with Efficiency)')
        plt.gca().invert_yaxis()
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"Error calculating mutual information: {e}")
        feature_importance = None
else:
    print("No suitable features found for importance analysis")
    feature_importance = None

In [ ]:
# Correlation analysis for feature selection
print("\nAnalyzing feature correlations...")

# Calculate correlations with efficiency
efficiency_correlations = df_features[feature_candidates + ['efficiency']].corr()['efficiency'].abs().sort_values(ascending=False)

print("Top 15 Features Correlated with Efficiency:")
for feature, corr in efficiency_correlations.head(16).items():  # 16 to exclude efficiency itself
    if feature != 'efficiency':
        sign = '+' if df_features[feature_candidates + ['efficiency']].corr()['efficiency'][feature] > 0 else '-'
        print(f"  {feature}: {sign}{corr:.3f}")

# Check for high correlation between features (multicollinearity)
feature_corr_matrix = df_features[feature_candidates].corr().abs()

# Find highly correlated feature pairs
high_corr_pairs = []
for i in range(len(feature_corr_matrix.columns)):
    for j in range(i+1, len(feature_corr_matrix.columns)):
        corr_value = feature_corr_matrix.iloc[i, j]
        if corr_value > 0.8:  # High correlation threshold
            high_corr_pairs.append((
                feature_corr_matrix.columns[i],
                feature_corr_matrix.columns[j],
                corr_value
            ))

if high_corr_pairs:
    print(f"\nFound {len(high_corr_pairs)} highly correlated feature pairs (>0.8):")
    for feat1, feat2, corr in high_corr_pairs[:10]:  # Show top 10
        print(f"  {feat1} <-> {feat2}: {corr:.3f}")
else:
    print("\nNo highly correlated feature pairs found (>0.8)")

## 9. Categorical Feature Encoding

In [ ]:
# Prepare categorical features for modeling
print("Encoding categorical features...")

# Identify categorical features
categorical_features = [
    'machine', 'OperatorName', 'PartNumber', 'State',
    'shift', 'week_position', 'experience_category', 'part_popularity'
]

# Filter to only include features that exist in the dataset
categorical_features = [f for f in categorical_features if f in df_features.columns]

print(f"Encoding {len(categorical_features)} categorical features: {categorical_features}")

# Create encoders dictionary
encoders = {}
encoded_features = df_features.copy()

for feature in categorical_features:
    if feature in encoded_features.columns:
        # Create label encoder
        le = LabelEncoder()
        
        # Handle missing values by filling with 'Unknown'
        encoded_features[feature] = encoded_features[feature].fillna('Unknown')
        
        # Encode the feature
        encoded_features[f'{feature}_encoded'] = le.fit_transform(encoded_features[feature].astype(str))
        
        # Store encoder for later use
        encoders[feature] = le
        
        # Print encoding info
        unique_values = len(le.classes_)
        print(f"  {feature}: {unique_values} unique values encoded")

print(f"\nCategorical encoding complete. Dataset shape: {encoded_features.shape}")

## 10. Final Feature Preparation

In [ ]:
# Prepare final feature set for modeling
print("Preparing final feature set...")

# Select modeling features (exclude raw categorical and identifier columns)
exclude_for_modeling = [
    'StartTime', 'EndTime', 'JobNumber', 'OpNumber', 'EmpID',
    'machine', 'OperatorName', 'PartNumber', 'State',  # Raw categorical (we have encoded versions)
    'shift', 'week_position', 'experience_category', 'part_popularity'  # Raw categorical
]

modeling_features = [col for col in encoded_features.columns if col not in exclude_for_modeling]

# Create final modeling dataset
df_modeling = encoded_features[modeling_features].copy()

print(f"Final modeling dataset shape: {df_modeling.shape}")
print(f"Features for modeling: {len(modeling_features)}")

# Handle any remaining missing values
missing_counts = df_modeling.isnull().sum()
features_with_missing = missing_counts[missing_counts > 0]

if len(features_with_missing) > 0:
    print(f"\nFeatures with missing values:")
    for feature, count in features_with_missing.items():
        print(f"  {feature}: {count} ({count/len(df_modeling)*100:.1f}%)")
    
    # Fill missing values with median for numerical, mode for categorical
    for feature in features_with_missing.index:
        if df_modeling[feature].dtype in ['int64', 'float64']:
            df_modeling[feature] = df_modeling[feature].fillna(df_modeling[feature].median())
        else:
            df_modeling[feature] = df_modeling[feature].fillna(df_modeling[feature].mode()[0])
    
    print("Missing values filled.")
else:
    print("No missing values found in final dataset.")

# Data quality check
print(f"\nFinal dataset validation:")
print(f"  Shape: {df_modeling.shape}")
print(f"  Missing values: {df_modeling.isnull().sum().sum()}")
print(f"  Infinite values: {np.isinf(df_modeling.select_dtypes(include=[np.number])).sum().sum()}")
print(f"  Efficiency range: {df_modeling['efficiency'].min():.3f} - {df_modeling['efficiency'].max():.3f}")

In [ ]:
# Feature categories summary
print("\nFEATURE CATEGORIES SUMMARY")
print("=" * 50)

feature_categories = {
    'Basic Performance': [f for f in modeling_features if f in ['efficiency', 'total_downtime', 'parts_per_hour']],
    'Time Features': [f for f in modeling_features if any(t in f for t in ['hour', 'day', 'time', 'shift', 'week'])],
    'Machine Features': [f for f in modeling_features if 'machine' in f],
    'Operator Features': [f for f in modeling_features if 'operator' in f],
    'Part Features': [f for f in modeling_features if 'part' in f],
    'Rolling/Trend Features': [f for f in modeling_features if 'roll' in f or 'trend' in f],
    'Ratio Features': [f for f in modeling_features if 'ratio' in f or 'intensity' in f],
    'Anomaly Features': [f for f in modeling_features if 'extreme' in f or 'rush' in f],
    'Quality Features': [f for f in modeling_features if any(q in f for q in ['consistency', 'stability', 'deviation'])],
    'Encoded Categorical': [f for f in modeling_features if '_encoded' in f]
}

for category, features in feature_categories.items():
    if features:
        print(f"\n{category} ({len(features)} features):")
        for feature in features[:5]:  # Show first 5
            print(f"  - {feature}")
        if len(features) > 5:
            print(f"  ... and {len(features) - 5} more")

total_categorized = sum(len(features) for features in feature_categories.values())
uncategorized = len(modeling_features) - total_categorized

print(f"\nTotal categorized features: {total_categorized}")
print(f"Uncategorized features: {uncategorized}")
print(f"Total modeling features: {len(modeling_features)}")

## 11. Save Processed Data

In [ ]:
# Save the processed datasets
import os

# Create output directory
output_dir = '../data/processed'
os.makedirs(output_dir, exist_ok=True)

# Save datasets
print("Saving processed datasets...")

# Save feature-engineered dataset
features_path = os.path.join(output_dir, 'features_engineered.csv')
encoded_features.to_csv(features_path, index=False)
print(f"Feature-engineered dataset saved: {features_path}")

# Save modeling-ready dataset
modeling_path = os.path.join(output_dir, 'modeling_ready.csv')
df_modeling.to_csv(modeling_path, index=False)
print(f"Modeling-ready dataset saved: {modeling_path}")

# Save feature information
feature_info = {
    'total_features': len(modeling_features),
    'feature_categories': {cat: len(feats) for cat, feats in feature_categories.items() if feats},
    'categorical_encoders': list(encoders.keys()),
    'data_shape': df_modeling.shape,
    'efficiency_stats': {
        'mean': df_modeling['efficiency'].mean(),
        'std': df_modeling['efficiency'].std(),
        'min': df_modeling['efficiency'].min(),
        'max': df_modeling['efficiency'].max()
    }
}

import json
info_path = os.path.join(output_dir, 'feature_info.json')
with open(info_path, 'w') as f:
    json.dump(feature_info, f, indent=2, default=str)
print(f"Feature information saved: {info_path}")

# Save encoders
import pickle
encoders_path = os.path.join(output_dir, 'categorical_encoders.pkl')
with open(encoders_path, 'wb') as f:
    pickle.dump(encoders, f)
print(f"Categorical encoders saved: {encoders_path}")

print("\nAll processed data saved successfully!")

## Summary

### Feature Engineering Complete!

This notebook has successfully created a comprehensive set of engineered features from the raw CNC data:

#### Features Created:
- **Time-based features**: Hour, day, shift patterns, rush hours, sequence numbers
- **Performance ratios**: Efficiency metrics, productive time ratios, downtime intensity
- **Historical features**: Machine and operator historical performance
- **Rolling features**: Trend analysis with multiple window sizes
- **Interaction features**: Machine-operator experience combinations
- **Part complexity**: Setup complexity, popularity, versatility scores
- **Anomaly indicators**: Extreme value detection, quality flags
- **Categorical encoding**: Label-encoded categorical variables

#### Key Outputs:
1. **Feature-engineered dataset**: Complete dataset with all derived features
2. **Modeling-ready dataset**: Clean dataset ready for ML training
3. **Feature importance analysis**: Top predictive features identified
4. **Categorical encoders**: Saved for consistent encoding in production

#### Next Steps:
1. **Model Development**: Use the prepared features to train ML models
2. **Feature Selection**: Further refine feature set based on model performance
3. **Hyperparameter Tuning**: Optimize models using the engineered features
4. **Performance Validation**: Test models on hold-out data

The feature engineering process has created a rich, informative dataset that should enable accurate predictions of efficiency, downtime, and optimal operator assignments.